In [1]:
import auto_diff
import numpy as np

In [2]:
def align_pi_plane_with_axes_rot():
    """
    Returns a matrix that rotates the pi plane's normal to be the z axis
    i.e., a slice of pi plane becomes the xy plane after rotation
    """
    pi_vector = np.array([1, 1, 1]) / np.sqrt(3.)
    # wanted_vector = np.array([1, 0, 0])
    wanted_vector = np.array([0, 0, 1])
    wanted_vector = wanted_vector / np.linalg.norm(wanted_vector)
    added = (pi_vector + wanted_vector).reshape([-1, 1])
    # from Rodrigues' rotation formula, more info here: https://math.stackexchange.com/a/2672702
    rot_mat = 2 * (added @ added.T) / (added.T @ added) - np.eye(3)
    return rot_mat


def align_axes_with_pi_plane_rot():
    """
    Returns a matrix that undoes the align_pi_plane_with_axes_rot rotation
    """
    return np.linalg.inv(align_pi_plane_with_axes_rot())

align_axes_with_pi_plane_rot = align_axes_with_pi_plane_rot()
align_pi_plane_with_axes_rot = align_pi_plane_with_axes_rot()

def A_mapping(real_eqps):

    # real_eqps = 0
    # fict_eqps = 0
    B_in = ((np.array(0.19468542))/(real_eqps + np.array(0.19468542)) ) * (np.array([[-0.78883927, -0.17774928,  0.05046644],
       [-0.94038756, -0.01281956,  0.0229076 ],
       [-0.80953323, -0.10425625, -0.02368224]]))
    B_in = align_axes_with_pi_plane_rot.T @ B_in @ align_axes_with_pi_plane_rot
    B_in[:, 2] = 0
    B_in[2, :] = 0
    B_in[2, 2] = 1
    B_in = align_axes_with_pi_plane_rot @ B_in @ align_axes_with_pi_plane_rot.T
    new_B = np.zeros((6,6))
    new_B[0:3, 0:3] = B_in
    new_B[3:, 3:] = np.eye(3) # Like the stuffness matrix, the shear comps are multiplied by ROOT2

    return new_B

P_vm = np.array([[1, -0.5, -0.5, 0, 0, 0],
                [-0.5, 1, -0.5, 0, 0, 0],
                [-0.5, -0.5, 1, 0, 0, 0],
                [0, 0, 0, 3, 0, 0],
                [0, 0, 0, 0, 3, 0],
                [0, 0, 0, 0, 0, 3]])

def yield_function_mandell(mandel_stress_vec, eqps ):
    A = A_mapping(eqps[0][0]) 
    G = np.transpose(mandel_stress_vec) @ ( np.transpose(A) @ P_vm @ A ) @ mandel_stress_vec

    return G - 1


In [4]:
x = np.array([[ 1, 2, 50000, 0, 0, 0 ]]).T
y = np.array([[ 0 ]]).T
print(y.shape)
print(x.shape)
with auto_diff.AutoDiff(x,y) as (x,y):
    f_eval = yield_function_mandell(x, y)
    Y, (Jx, Jy) = auto_diff.get_value_and_jacobians(f_eval)
print(Jx, Jy)

(1, 1)
(6, 1)
[[ -81.37552979 -267.02300223  348.39853202    0.            0.
     0.        ]] [[-89474143.36216609]]
